In [14]:
import pandas as pd
import requests
import json
from datetime import datetime
import pytz
import schedule

In [15]:

data_path="../../weatherman/"

In [16]:
def create_6days_list(no_of_days=6):
    """
    Create a list of timestamps with the first being the immediate timestamp and the other 5 for the 5 days in the future.

    Keyword arguments:
    no_of_days -- the total number of days you want timestamps for (default 6)
    """
    a_day=86400
    day0 = datetime.strftime(datetime.utcnow(), "%s")
    past_6_days=[]

    for i in range(no_of_days):
        one_day=int(day0) - int(a_day*i)
        past_6_days.append(one_day)
    return(past_6_days)

past_6_days_list=create_6days_list(6)
print(past_6_days_list)

[1645615499, 1645529099, 1645442699, 1645356299, 1645269899, 1645183499]


In [17]:
# infer_dst = np.array([False] * df.shape[0])  # all False -> every row considered DT, alternative is True to indicate DST. The array must correspond to the iloc of df.index
# df.index.tz_localize('Europe/Amsterdam', ambiguous=infer_dst)  # no error

NameError: name 'np' is not defined

In [23]:
def get_historical_weather(
    lat=51.17,
    lon=10.45,
    api_key="bc2d732248597a8725c1f5d51e5d3e8a",
    past_6_days=past_6_days_list,
):
    """
    Returns a dataframe with dt and temp columns.

    Keyword arguments:
    lat -- the latitude (default 51.17)
    lon -- the longitude (default 10.45)
    api_key -- api key gotten from openweathermap (default "bc2d732248597a8725c1f5d51e5d3e8a")
    past_6_days -- a list of timestamps for the 6 days you want historical data for
    """
    new_df = pd.DataFrame()
    past_6_days_df = pd.DataFrame()

    for day in past_6_days:
        url = (
            "https://api.openweathermap.org/data/2.5/onecall/timemachine?lat=%s&lon=%s&dt=%s&units=metric&appid=%s" % (lat, lon, day, api_key)
        )
        response = requests.get(url)
        data = json.loads(response.text)
        hourly_data = data["hourly"]

        for entry in hourly_data:
            temp = pd.DataFrame.from_dict(pd.json_normalize(entry), orient="columns")

            # select only nec. columns
            columns = ["dt", "temp"]
            temp = temp[columns]

            # transform dt column
            temp["dt"] = datetime.fromtimestamp(entry["dt"], pytz.utc)
            new_df = new_df.append(temp, ignore_index=True)

        # concatenate each day in a new dataframe
        past_6_days_df = pd.concat(
            [past_6_days_df, new_df], axis=0, ignore_index=False
        ).drop_duplicates()

    return(past_6_days_df)

get_historical_weather(api_key = "bc2d732248597a8725c1f5d51e5d3e8a",lat=51.17,lon=10.45)

,dt,temp
0,2022-02-23 00:00:00+00:00,5.86
1,2022-02-23 01:00:00+00:00,5.03
2,2022-02-23 02:00:00+00:00,4.49
3,2022-02-23 03:00:00+00:00,4.31
4,2022-02-23 04:00:00+00:00,4.84
...,...,...
128,2022-02-18 19:00:00+00:00,8.92
129,2022-02-18 20:00:00+00:00,8.39
130,2022-02-18 21:00:00+00:00,7.47
131,2022-02-18 22:00:00+00:00,6.91


In [5]:
# # Warning! Running this will overwrite current datasets

# germany=get_historical_weather(api_key = "bc2d732248597a8725c1f5d51e5d3e8a",lat=51.17,lon=10.45)
# germany.set_index(['dt']).to_csv(data_path + "germany_weather.csv")

# netherlands=get_historical_weather(api_key = "bc2d732248597a8725c1f5d51e5d3e8a",lat=52.13,lon=5.29)
# netherlands.set_index(['dt']).to_csv(data_path + "netherlands_weather.csv")

# uk=get_historical_weather(api_key = "bc2d732248597a8725c1f5d51e5d3e8a",lat=52.35,lon=-1.17)
# uk.set_index(['dt']).to_csv(data_path + "uk_weather.csv")

In [6]:
# Run this to append new entries

germany=get_historical_weather(api_key = "bc2d732248597a8725c1f5d51e5d3e8a",lat=51.17,lon=10.45)
pd.read_csv(data_path + "germany_weather.csv",index_col = False).set_index(['dt'])\
    .append(germany.set_index(['dt'])).drop_duplicates().to_csv(data_path + "germany_weather.csv")

netherlands=get_historical_weather(api_key = "bc2d732248597a8725c1f5d51e5d3e8a",lat=52.13,lon=5.29)
pd.read_csv(data_path + "netherlands_weather.csv",index_col = False).set_index(['dt'])\
    .append(netherlands.set_index(['dt'])).drop_duplicates().to_csv(data_path + "netherlands_weather.csv")

uk=get_historical_weather(api_key = "bc2d732248597a8725c1f5d51e5d3e8a",lat=52.35,lon=-1.17)
pd.read_csv(data_path + "uk_weather.csv",index_col = False).set_index(['dt'])\
    .append(uk.set_index(['dt'])).drop_duplicates().to_csv(data_path + "uk_weather.csv")

In [ ]:

# new_df=pd.DataFrame()
# past_6_days_df=pd.DataFrame()

# for day in past_6_days:
#     url="https://api.openweathermap.org/data/2.5/onecall/timemachine?lat=%s&lon=%s&dt=%s&units=metric&appid=%s" % (lat, lon, day, api_key)
#     response = requests.get(url)
#     data = json.loads(response.text)
#     hourly = data["hourly"]

#     for entry in hourly:
#         # temp = pd.DataFrame(entry)
#         temp=pd.DataFrame.from_dict(pd.json_normalize(entry), orient='columns')

#         # select only nec. columns
#         columns=['dt','temp']
#         temp=temp[columns]

#         # transform dt column
#         temp['dt']=datetime.fromtimestamp(entry["dt"], pytz.timezone(data['timezone']))
#         new_df=new_df.append(temp,ignore_index=True)

#     # print(new_df)

#     # concatenate each day in a new dataframe   
#     past_6_days_df=pd.concat([past_6_days_df,new_df],axis=0, ignore_index=False).drop_duplicates()

# print(past_6_days_df)

In [ ]:
# new_df=pd.DataFrame()
# daily = data["daily"]
# for entry in daily:
#     dt = datetime.fromtimestamp(entry["dt"], pytz.timezone(data['timezone']))
#     temp = entry["temp"]
#     temp['dt']=dt
#     new_df=new_df.append(temp,ignore_index=True)
# new_df['timezone']=data['timezone']
# new_df

# this works for forecast

In [ ]:
# def get_weather_forecast(lat,lon,api_key):
    
#     new_df=pd.DataFrame()
#     url="https://api.openweathermap.org/data/2.5/onecall?lat=%s&lon=%s&exclude=current,minutely,hourly,alerts&units=metric&appid=%s" % (lat, lon, api_key)
#     response = requests.get(url)
#     data = json.loads(response.text)
    

#     daily = data["daily"]
#     for entry in daily:
#         dt = datetime.fromtimestamp(entry["dt"], pytz.timezone(data['timezone']))
#         temp = entry["temp"]
#         temp['dt']=dt
#         new_df=new_df.append(temp,ignore_index=True)
#     new_df['timezone']=data['timezone']
#     return(new_df)


In [ ]:
# germany=get_weather_forecast(api_key = "bc2d732248597a8725c1f5d51e5d3e8a",lat=51.17,lon=10.45)
# germany

# netherlands=get_weather_forecast(api_key = "3359ae5263476eb42c0c2bfa08ecc6c3",lat=52.13,lon=5.29)
# netherlands

# uk=get_weather_forecast(api_key = "3359ae5263476eb42c0c2bfa08ecc6c3",lat=52.35,lon=-1.17)
# uk

In [ ]:
# df=pd.DataFrame.from_dict(pd.json_normalize(data), orient='columns')